In [50]:
import pandas as pd
import numpy as np

In [10]:
bigoldlist = ['2013-02.json',
'2013-03.json',
'2013-04.json',
'2013-05.json',
'2013-06.json',
'2013-07.json',
'2013-08.json',
'2013-09.json',
'2013-10.json',
'2013-11.json',
'2013-12.json',
'2014-01.json',
'2014-02.json',
'2014-03.json',
'2014-04.json',
'2014-05.json',
'2014-06.json',
'2014-07.json',
'2014-08.json',
'2014-09.json',
'2014-10.json',
'2014-11.json',
'2014-12.json',
'2015-01.json',
'2015-02.json',
'2015-03.json',
'2015-04.json',
'2015-05.json',
'2015-06.json',
'2015-07.json',
'2015-08.json',
'2015-09.json',
'2015-10.json',
'2015-11.json',
'2015-12.json',
'2016-01.json',
'2016-02.json',
'2016-03.json',
'2016-04.json',
'2016-05.json',
'2016-06.json',
'2016-07.json',
'2016-08.json',
'2016-09.json',
'2016-10.json',
'2016-11.json',
'2016-12.json',
'2017-01.json',
'2017-02.json',
'2017-03.json',
'2017-04.json',
'2017-05.json',
'2017-06.json',
'2017-07.json',
'2017-08.json',
'2017-09.json',
'2017-10.json',
'2017-11.json',
'2017-12.json',
'2018-01.json',
'2018-02.json',
'2018-03.json',
'2018-04.json',
'2018-05.json',
'2018-06.json',
'2018-07.json',
'2018-08.json',
'2018-09.json',
'2018-10.json',
'2018-11.json',
'2018-12.json',
'2019-01.json',
'2019-02.json',
'2019-03.json',
'2019-04.json',
'2019-05.json',
'2019-06.json',
'2019-07.json',
'2019-08.json',
'2019-09.json',
'2019-10.json',
'2019-11.json',
'2019-12.json',
'2020-01.json',
'2020-02.json',
'2020-03.json',
'2020-04.json',
'2020-05.json',
'2020-06.json',
'2020-07.json',
'2020-08.json',
'2020-09.json',
'2020-10.json',
'2020-11.json',
'2020-12.json',
'2021-01.json',
'2021-02.json',
'2021-03.json',
'2021-04.json']

In [13]:
# test
url = 'https://terra.snellman.net/data/events/' + bigoldlist[0]
eventsdata1 = pd.read_json(url)
print(len(eventsdata1))
eventsdata1.head()

1


,last_update,events,player_count,factions,base_map,game
0,2013-02-10 02:40:12,{'global': {'faction-count': {'round': {'all':...,3,"[{'player': None, 'faction': 'darklings'}, {'p...",126fe960806d587c78546b30f1a90853b1ada468,pbc3


In [56]:
# make dataframe
factions = ['witches', 'aurens', 'swarmlings', 'mermaids', 'cultists', 'halflings', 'dwarves', 'engineers', 'chaosmagicians', 'giants', 'fakirs', 'nomads', 'darklings', 'alchemists']
dfcols = ['game'] + factions
vpdf = pd.DataFrame(columns=dfcols)
newdf = pd.DataFrame([[np.nan] * 15], columns=dfcols)
newdf['aurens'].replace({np.nan: 'test'}, inplace=True)
vpdf = vpdf.append(newdf, ignore_index=True)
print(len(vpdf))
vpdf.head()

1


,game,witches,aurens,swarmlings,mermaids,cultists,halflings,dwarves,engineers,chaosmagicians,giants,fakirs,nomads,darklings,alchemists
0,NaN,NaN,test,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# load in the data
for gameset in bigoldlist:
    url = 'https://terra.snellman.net/data/events/' + gameset
    eventsdata = pd.read_json(url)
    
    for eachgame in range(len(gameset)):
        singlerow = eventsdata.iloc[[eachgame]]
        newcol = get_vp_from_game(gameEventRow)
        vpdf = vpdf.append(newdf, ignore_index=True)      

In [ ]:
def get_vp_from_game(gameEventRow):
    """Input game event row from data/events. Output a row where each faction in the game has its vp populated (the rest are nans)
    """
    newdf = pd.DataFrame([[np.nan] * 15], columns=dfcols)
    
    for faction in gamefactions:
        vpfaction = singlegame[(singlegame['event'] == 'vp') & (singlegame['faction'] == faction)]
        newdf[faction].replace({np.nan: vpfaction}, inplace=True)